In [1]:
from skimage import io
import napari
from brainlit.preprocessing.image_process import compute_frags, split_frags, remove_somas, label_points
from brainlit.algorithms import dynamic_programming_viterbi2
import h5py
from skimage import measure
import numpy as np
import time

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
im = io.imread("/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_0_first10_quantitative.tif")
scale = [0.3, 0.3, 1]
soma_coords = [[360, 240, 120]]
threshold = 0.9

pred_path ="/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/2018-08-01_0_first10_quantitative_Probabilities.h5"
f = h5py.File(pred_path, 'r')
pred = f.get('exported_data')
pred = pred[:,:,:,1]


mask = pred > threshold
labels = measure.label(mask)

In [3]:
t1 = time.perf_counter()
new_labels = compute_frags(soma_coords=soma_coords, labels=labels, im_processed = pred, threshold = threshold, res=scale, chunk_size=[200,200,200], ncpu=8)
#new_labels2 = split_frags(soma_coords=soma_coords, labels=labels, im_processed = pred, threshold = threshold, res=scale)
print(time.perf_counter()-t1)

Parallelizing with 8 cpus


/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
removing somas: 0it [00:00, ?it/s]
looking for components to remove:  25%|██▍       | 32/130 [00:00<00:00, 304.69it/s]/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
removing somas: 0it [00:00, ?it/s]
looking for components to remove: 100%|██████████| 130/130 [00:00<00:00, 277.28it/s]
/Users/thomasathey/Documents/mimlab/mouselight/docs_en

15.018212747000007


In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(im, name="image", scale=scale)
viewer.add_labels(labels, name="labels", scale=scale)
viewer.add_labels(new_labels, name="labels", scale=scale)
#viewer.add_labels(new_labels2, name="labels", scale=scale)

In [4]:
_, soma_lbls = label_points(new_labels, soma_coords, scale)
t1 = time.perf_counter()
mpnp = dynamic_programming_viterbi2.most_probable_neuron_path(image=im.astype(float), labels=new_labels, soma_lbls=soma_lbls, resolution=scale, coef_dist=10, coef_curv=1000)
print(time.perf_counter()-t1)


Setting up emission distribution...
18.013899616000003


In [6]:
mpnp.frags_to_lines(ncpu=8)

Parallelizing x8


In [ ]:
t1 = time.perf_counter()
mpnp.reset_dists(type="all")
mpnp.compute_all_costs_dist_par(point_point_func=mpnp.point_point_dist, point_blob_func=mpnp.point_blob_dist, ncpu=1)
print(time.perf_counter()-t1)

Normalizing: 100%|██████████| 3361/3361 [00:00<00:00, 17488.56it/s]

289.62475736299996


smart pairs=174s

all pairs =289s